In [3]:
"""
Cryptocurrency Trading Strategy with GPR+LSTM Model

This program implements and backtests a trading strategy for cryptocurrencies
using a combination of Gaussian Process Regression with changepoint detection
and LSTM neural networks.

Author: agehcx
Date: 2025-03-07
"""

import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime
import ruptures as rpt
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)

#------------------------------------------------------------------------------
# CONFIGURATION
#------------------------------------------------------------------------------

# Model hyperparameters
MODEL_PARAMS = {
    'batch_size': 128,
    'dropout_rate': 0.2,
    'learning_rate': 0.001,
    'lookback_window': 21,
    'lstm_hidden_units': 40,
    'epochs': 50,
    'train_ratio': 0.8
}

# Trading parameters
TRADING_PARAMS = {
    'fee_rate': 0.001,  # 0.1% fee per trade
    'position_size': 1.0,  # Full position size
    'frequency': '1D'     # Daily rebalancing
}

# Dictionary of top cryptocurrencies by year with -USD suffix
TOP_CRYPTOS = {
    "2019": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "EOS-USD", "LTC-USD", "XLM-USD", "ADA-USD", "TRX-USD", "BSV-USD"],
    "2020": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "LTC-USD", "EOS-USD", "BNB-USD", "BSV-USD", "ADA-USD", "XTZ-USD"],
    "2021": ["BTC-USD", "ETH-USD", "XRP-USD", "LTC-USD", "BCH-USD", "ADA-USD", "DOT-USD", "LINK-USD", "BNB-USD", "XLM-USD"],
    "2022": ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "ADA-USD", "XRP-USD", "DOT-USD", "LUNA-USD", "AVAX-USD", "DOGE-USD"],
    "2023": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "MATIC-USD", "DOT-USD", "LTC-USD", "SHIB-USD"],
    "2024": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"],
    "2025": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT-USD", "LTC-USD", "AVAX-USD"]
}

# Output file
RESULTS_FILE = "Backtested/vol_weight.csv"

#------------------------------------------------------------------------------
# DATA PROCESSING
#------------------------------------------------------------------------------

def fetch_and_process_data(tickers, start_date, end_date):
    """
    Fetch data for multiple tickers and return price and standardized returns DataFrames.
    
    Args:
        tickers (list): List of ticker symbols to fetch data for
        start_date (str): Start date in YYYY-MM-DD format
        end_date (str): End date in YYYY-MM-DD format
        
    Returns:
        tuple: (prices_df, returns_df) - DataFrames containing price data and standardized returns
    """
    price_data = {}
    returns_data = {}
    
    for ticker in tickers:
        try:
            # Download data from Yahoo Finance
            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty:
                print(f"No data found for {ticker} between {start_date} and {end_date}")
                continue
                
            # Prefer "Adj Close" if available, otherwise "Close"
            if "Adj Close" in data.columns:
                price_series = data["Adj Close"].copy()
            else:
                price_series = data["Close"].copy()
                
            # Ensure we have a series, not a DataFrame
            if isinstance(price_series, pd.DataFrame):
                price_series = price_series.squeeze()
                
            price_series.name = ticker
            
            # Calculate returns and drop any NA values
            returns = price_series.pct_change().dropna()
            
            # Standardize returns for better model performance
            scaler = StandardScaler()
            std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
            valid_index = returns.index
            
            price_data[ticker] = price_series.loc[valid_index]
            returns_data[ticker] = pd.Series(
                std_returns.flatten(), index=valid_index, name=ticker
            )
            
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
    
    # Convert to DataFrames
    prices_df = pd.DataFrame({ticker: series for ticker, series in price_data.items()})
    returns_df = pd.DataFrame({ticker: series for ticker, series in returns_data.items()})
    
    return prices_df, returns_df

#------------------------------------------------------------------------------
# CHANGEPOINT DETECTION
#------------------------------------------------------------------------------

class ChangePointKernel(Kernel):
    """
    Custom kernel for Gaussian Process Regression that accounts for regime changes.
    
    This kernel multiplies a base kernel with a mask that's 1 when two points are in the
    same regime (separated by changepoints) and 0 otherwise.
    """
    def __init__(self, base_kernel, changepoints):
        """
        Initialize the ChangePointKernel.
        
        Args:
            base_kernel: The base kernel (e.g., Matern32)
            changepoints (list): Indices where regime changes are detected
        """
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected
        self.changepoints = changepoints

    def get_region(self, X):
        """
        Determine the region (between which changepoints) each data point falls.
        
        Args:
            X: Input tensor
            
        Returns:
            Tensor containing the region index for each input point
        """
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        """
        Compute covariance matrix between inputs X and X2.
        
        Args:
            X: First input
            X2: Second input (defaults to X if None)
            
        Returns:
            Covariance matrix
        """
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        # Create a mask that's 1 when points are in the same region, 0 otherwise
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        # Get the base kernel's covariance matrix
        base_cov = self.base_kernel.K(X, X2)
        # Apply the mask to the base covariance
        return base_cov * regions_equal

    def K_diag(self, X):
        """
        Compute diagonal of covariance matrix for inputs X.
        
        Args:
            X: Input tensor
            
        Returns:
            Diagonal of covariance matrix
        """
        return self.base_kernel.K_diag(X)


def detect_changepoints(returns, lookback_window):
    """
    Detect changepoints in a return series using Pelt algorithm with RBF model.
    
    Args:
        returns (pd.Series): Series of returns to analyze
        lookback_window (int): Window size to consider for changepoint detection
        
    Returns:
        list: Indices where changepoints are detected
    """
    # Use ruptures to detect changepoints using an RBF model
    algo = rpt.Pelt(model="rbf").fit(returns.values)
    
    # The penalty parameter controls the number of changepoints
    # Using the lookback_window as penalty is a simplified approach
    result = algo.predict(pen=lookback_window)
    
    return result

#------------------------------------------------------------------------------
# MODEL IMPLEMENTATION
#------------------------------------------------------------------------------

def generate_signals(returns, changepoints, train_ratio, epochs, 
                     dropout_rate, lstm_hidden_units, learning_rate, batch_size):
    """
    Generate trading signals using combined GPR+LSTM model.
    
    Args:
        returns (pd.Series): Standardized returns to model
        changepoints (list): Detected changepoints in the series
        train_ratio (float): Ratio of data to use for training (0.0-1.0)
        epochs (int): Number of training epochs for LSTM
        dropout_rate (float): Dropout rate for LSTM
        lstm_hidden_units (int): Number of hidden units in LSTM layer
        learning_rate (float): Learning rate for Adam optimizer
        batch_size (int): Batch size for training
        
    Returns:
        numpy.ndarray: Array of position signals (-1 for short, 1 for long)
    """
    # Use time index as a feature
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use Gaussian Process Regression with the ChangePointKernel for trend extraction
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    
    # Extract the trend component using GPR
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form a feature set
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    split_index = int(len(features) * train_ratio)
    
    # Split into training and testing data
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape features for LSTM input (samples, timesteps, features)
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    
    # Build the LSTM model
    model = Sequential([
        # First LSTM layer returns sequences for stacking
        LSTM(lstm_hidden_units, return_sequences=True,
             input_shape=(train_features.shape[1], train_features.shape[2])),
        Dropout(dropout_rate),  # Apply dropout to prevent overfitting
        # Second LSTM layer
        LSTM(int(lstm_hidden_units / 2)),
        # Output layer with tanh activation (outputs between -1 and 1)
        Dense(1, activation="tanh")
    ])
    
    # Compile the model with Adam optimizer and MSE loss
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    
    # Train the model
    model.fit(train_features, train_labels, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Predict signals for all features
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Convert signals to binary (-1 or 1) for direction only
    # For equal weighting, we only care about direction (long/short)
    positions = np.sign(predicted_signals)
    positions[positions == 0] = 1  # Convert any zeros to 1 (long)
    
    return positions

#------------------------------------------------------------------------------
# BACKTESTING
#------------------------------------------------------------------------------

def backtest_strategy_single_asset(price_series, positions, fee_rate=0.001):
    """
    Backtest a trading strategy for a single asset.
    
    Args:
        price_series (pd.Series): Series of price data
        positions (pd.Series/array): Array of position signals (-1 for short, 1 for long)
        fee_rate (float): Fee rate as a decimal (default: 0.001 or 0.1%)
        
    Returns:
        vectorbt.Portfolio: Portfolio object containing backtest results
    """
    # Ensure price_series is a Series, not a DataFrame
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    # Convert positions to a Series with the same index as price_series
    # positions = pd.Series(positions, index=price_series.index, name=price_series.name) ## Old 
    positions = pd.Series(positions, index=price_series.index[:len(positions)], name=price_series.name) ## New
    
    # Generate entry/exit signals from positions
    entries = positions > 0  # Long signals
    exits = positions < 0    # Short signals
    
    # Create portfolio using vectorbt
    pf = vbt.Portfolio.from_signals(
        close=price_series,
        entries=entries,
        exits=exits,
        # signal=positions2, # Use positions as signals instead of entries/exits
        size=1.0,  # Always use full position size
        freq="1D",  # Daily data
        fees=fee_rate,
    )
    return pf


def backtest_equal_weight_portfolio(prices_df, signals_dict, fee_rate=0.001):
    """
    Backtest a portfolio with equal weights across assets.
    
    Args:
        prices_df (pd.DataFrame): DataFrame of price data for multiple assets
        signals_dict (dict): Dictionary mapping asset names to position signals
        fee_rate (float): Fee rate for transactions
        
    Returns:
        tuple: (portfolio, asset_portfolios) - Portfolio object and dict of individual asset portfolios
    """
    # Create a dictionary to store individual asset performance
    asset_results = {}
    
    for ticker in prices_df.columns:
        if ticker in signals_dict:
            try:
                # Get price series and signals for this ticker
                price_series = prices_df[ticker]
                positions = signals_dict[ticker]
                
                # Make sure positions align with price index
                positions_series = pd.Series(positions, index=price_series.index[:len(positions)])
                
                # Backtest the individual asset
                pf = backtest_strategy_single_asset(price_series, positions_series, fee_rate)
                asset_results[ticker] = pf
            except Exception as e:
                print(f"Error backtesting {ticker}: {e}")
    
    # Calculate the equal weight allocation (1/n for each asset)
    n_assets = len(asset_results)
    if n_assets == 0:
        return None, {}
        
    weight_per_asset = 1.0 / n_assets
    
    # Create a combined DataFrame for returns
    combined_returns = pd.DataFrame()
    
    # Extract returns from each asset portfolio
    for ticker, pf in asset_results.items():
        returns = pf.returns()
        if not returns.empty:
            combined_returns[ticker] = returns
    
    # Calculate equal-weighted portfolio returns
    if not combined_returns.empty:
        # Replace NaN with 0 for calculation
        combined_returns.fillna(0, inplace=True)
        portfolio_returns = (combined_returns * weight_per_asset).sum(axis=1)
        
        # Calculate cumulative returns
        portfolio_cumulative = (1 + portfolio_returns).cumprod()
        
        # Create a mock portfolio object with key stats
        portfolio_stats = {
            "total_return": portfolio_cumulative.iloc[-1] - 1 if len(portfolio_cumulative) > 0 else 0,
            "sharpe_ratio": portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 0 else 0,
            "max_drawdown": (portfolio_cumulative / portfolio_cumulative.cummax() - 1).min() if len(portfolio_cumulative) > 0 else 0,
            "win_rate": np.sum(portfolio_returns > 0) / len(portfolio_returns) if len(portfolio_returns) > 0 else 0,
        }
        
        # Create mock portfolio object with stats method
        mock_portfolio = type('obj', (object,), {
            'stats': lambda: portfolio_stats
        })
        
        return mock_portfolio, asset_results
    else:
        return None, {}

def calculate_volatility_weights(tickers, current_year, lookback_months=12):
    """
    Calculate volatility-adjusted weights for a portfolio of cryptocurrencies.
    
    Args:
        tickers (list): List of cryptocurrency tickers
        current_year (int): The current year for the portfolio
        lookback_months (int): Number of months to look back for volatility calculation
        
    Returns:
        dict: Dictionary of volatility-adjusted weights for each ticker
    """
    # Define the lookback period for volatility calculation
    end_date = f"{current_year}-01-01"  # Start of current year
    
    # Calculate start date based on lookback_months
    from dateutil.relativedelta import relativedelta
    from datetime import datetime
    
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
    start_date_dt = end_date_dt - relativedelta(months=lookback_months)
    start_date = start_date_dt.strftime("%Y-%m-%d")
    
    print(f"Calculating volatility from {start_date} to {end_date}")
    
    # Fetch historical data for volatility calculation
    volatilities = {}
    valid_tickers = []
    
    for ticker in tickers:
        try:
            # Download data from Yahoo Finance
            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty or len(data) < 30:  # Require at least 30 days of data
                print(f"Insufficient historical data for {ticker}, using default weight")
                continue
                
            # Calculate daily returns
            if "Adj Close" in data.columns:
                returns = data["Adj Close"].pct_change().dropna()
            else:
                returns = data["Close"].pct_change().dropna()
                
            # Calculate annualized volatility (standard deviation of returns * sqrt(252))
            annual_vol = returns.std() * np.sqrt(252)
            volatilities[ticker] = annual_vol
            valid_tickers.append(ticker)
            
            print(f"{ticker} annual volatility: {annual_vol:.4f}")
            
        except Exception as e:
            print(f"Error calculating volatility for {ticker}: {e}")
    
    # Calculate inverse volatility weights
    if not volatilities:
        print("No valid volatility data, returning equal weights")
        return {ticker: 1.0/len(tickers) for ticker in tickers}
    
    # Calculate inverse of volatility (lower volatility = higher weight)
    inverse_volatilities = {ticker: 1.0/vol for ticker, vol in volatilities.items()}
    
    # Normalize weights to sum to 1.0
    total_inverse_vol = sum(inverse_volatilities.values())
    weights = {ticker: inv_vol/total_inverse_vol for ticker, inv_vol in inverse_volatilities.items()}
    
    # For tickers with no volatility data, assign minimum weight
    min_weight = min(weights.values()) if weights else 0
    for ticker in tickers:
        if ticker not in weights:
            weights[ticker] = min_weight
    
    # Re-normalize to ensure sum is exactly 1.0
    total_weight = sum(weights.values())
    normalized_weights = {ticker: weight/total_weight for ticker, weight in weights.items()}
    
    return normalized_weights

def calculate_volatility_weights(tickers, current_year, lookback_months=12):
    """
    Calculate volatility-adjusted weights for a portfolio of cryptocurrencies.
    
    Args:
        tickers (list): List of cryptocurrency tickers
        current_year (int): The current year for the portfolio
        lookback_months (int): Number of months to look back for volatility calculation
        
    Returns:
        dict: Dictionary of volatility-adjusted weights for each ticker
    """
    # Define the lookback period for volatility calculation
    end_date = f"{current_year}-01-01"  # Start of current year
    
    # Calculate start date based on lookback_months
    from dateutil.relativedelta import relativedelta
    from datetime import datetime
    
    end_date_dt = datetime.strptime(end_date, "%Y-%m-%d")
    start_date_dt = end_date_dt - relativedelta(months=lookback_months)
    start_date = start_date_dt.strftime("%Y-%m-%d")
    
    print(f"Calculating volatility from {start_date} to {end_date}")
    
    # Fetch historical data for volatility calculation
    volatilities = {}
    valid_tickers = []
    
    for ticker in tickers:
        try:
            # Download data from Yahoo Finance
            data = yf.download(ticker, start=start_date, end=end_date)
            if data.empty or len(data) < 30:  # Require at least 30 days of data
                print(f"Insufficient historical data for {ticker}, using default weight")
                continue
                
            # Calculate daily returns
            if "Adj Close" in data.columns:
                returns = data["Adj Close"].pct_change().dropna()
            else:
                returns = data["Close"].pct_change().dropna()
                
            # Calculate annualized volatility (standard deviation of returns * sqrt(252))
            annual_vol = float(returns.std() * np.sqrt(252))  # Explicitly convert to float
            volatilities[ticker] = annual_vol
            valid_tickers.append(ticker)
            
            print(f"{ticker} annual volatility: {annual_vol:.4f}")
            
        except Exception as e:
            print(f"Error calculating volatility for {ticker}: {e}")
    
    # Calculate inverse volatility weights
    if len(volatilities) == 0:
        print("No valid volatility data, returning equal weights")
        return {ticker: 1.0/len(tickers) for ticker in tickers}
    
    # Calculate inverse of volatility (lower volatility = higher weight)
    inverse_volatilities = {ticker: 1.0/vol for ticker, vol in volatilities.items()}
    
    # Normalize weights to sum to 1.0
    total_inverse_vol = sum(inverse_volatilities.values())
    weights = {ticker: inv_vol/total_inverse_vol for ticker, inv_vol in inverse_volatilities.items()}
    
    # For tickers with no volatility data, assign minimum weight
    # Fix: Use explicit list conversion to avoid pandas Series behavior
    weight_values = list(weights.values())
    min_weight = min(weight_values) if weight_values else 0
    
    for ticker in tickers:
        if ticker not in weights:
            weights[ticker] = min_weight
    
    # Re-normalize to ensure sum is exactly 1.0
    total_weight = sum(weights.values())
    normalized_weights = {ticker: weight/total_weight for ticker, weight in weights.items()}
    
    return normalized_weights


def backtest_volatility_adjusted_portfolio(prices_df, signals_dict, current_year, fee_rate=0.001):
    """
    Backtest a portfolio with volatility-adjusted weights across assets.
    
    Args:
        prices_df (pd.DataFrame): DataFrame of price data for multiple assets
        signals_dict (dict): Dictionary mapping asset names to position signals
        current_year (int): Current year for the portfolio
        fee_rate (float): Fee rate for transactions
        
    Returns:
        tuple: (portfolio, asset_portfolios) - Portfolio object and dict of individual asset portfolios
    """
    # Create a dictionary to store individual asset performance
    asset_results = {}
    
    # Get list of valid tickers
    valid_tickers = [ticker for ticker in prices_df.columns if ticker in signals_dict]
    
    if not valid_tickers:
        print("No valid tickers found for volatility-adjusted portfolio")
        return None, {}
    
    # Calculate volatility-based weights
    weights = calculate_volatility_weights(valid_tickers, current_year)
    
    print(f"\nVolatility-adjusted weights:")
    for ticker, weight in sorted(weights.items(), key=lambda x: x[1], reverse=True):
        print(f"{ticker}: {weight:.4f}")
    
    # Process each asset
    for ticker in valid_tickers:
        try:
            # Get price series and signals for this ticker
            price_series = prices_df[ticker]
            positions = signals_dict[ticker]
            
            # Make sure positions align with price index
            positions_series = pd.Series(positions, index=price_series.index[:len(positions)])
            
            # Backtest the individual asset
            pf = backtest_strategy_single_asset(price_series, positions_series, fee_rate)
            asset_results[ticker] = pf
        except Exception as e:
            print(f"Error backtesting {ticker}: {e}")
    
    # Create a combined DataFrame for returns
    combined_returns = pd.DataFrame()
    
    # Extract returns from each asset portfolio
    for ticker, pf in asset_results.items():
        returns = pf.returns()
        if not returns.empty:
            combined_returns[ticker] = returns
    
    # Calculate weighted portfolio returns
    if not combined_returns.empty:
        # Replace NaN with 0 for calculation
        combined_returns.fillna(0, inplace=True)
        
        # Calculate weighted returns using volatility-based weights
        weighted_returns = pd.DataFrame()
        for ticker in combined_returns.columns:
            weight = weights.get(ticker, 0)
            weighted_returns[ticker] = combined_returns[ticker] * weight
        
        portfolio_returns = weighted_returns.sum(axis=1)
        
        # Calculate cumulative returns
        portfolio_cumulative = (1 + portfolio_returns).cumprod()
        
        # Create portfolio stats
        portfolio_stats = {
            "total_return": portfolio_cumulative.iloc[-1] - 1 if len(portfolio_cumulative) > 0 else 0,
            "sharpe_ratio": portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 0 else 0,
            "max_drawdown": (portfolio_cumulative / portfolio_cumulative.cummax() - 1).min() if len(portfolio_cumulative) > 0 else 0,
            "win_rate": np.sum(portfolio_returns > 0) / len(portfolio_returns) if len(portfolio_returns) > 0 else 0,
        }
        
        # Create mock portfolio object with stats method
        mock_portfolio = type('obj', (object,), {
            'stats': lambda: portfolio_stats
        })
        
        return mock_portfolio, asset_results
    else:
        return None, {}

#------------------------------------------------------------------------------
# MAIN FUNCTION
#------------------------------------------------------------------------------

def main():
    """
    Main function to execute the cryptocurrency trading strategy.
    
    This function:
    1. Defines the test periods
    2. Processes data for each period
    3. Generates signals using the GPR+LSTM model
    4. Backtests both equal-weight and volatility-adjusted portfolio strategies
    5. Saves and displays the results
    """
    # Define periods (one per year)
    periods = []
    for year in range(2019, 2025):
        period_label = str(year)
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        periods.append((period_label, start_date, end_date))
        
    # YTD 2025 using current date
    periods.append(("2025_YTD", "2025-01-01", "2025-03-18"))  # Updated to current date

    results = []

    # Create a separate file for volatility-adjusted results
    VOL_RESULTS_FILE = "Backtested/volatility_adjusted_portfolio_results.csv"

    # Loop over each period and backtest the respective top cryptocurrencies for that year
    for period_label, start_date, end_date in periods:
        year = period_label.split("_")[0]  # Extract year from period label
        current_year = int(year)
        cryptos_for_year = TOP_CRYPTOS.get(year, [])
        
        print(f"\nProcessing period {period_label} ({start_date} to {end_date})")
        print(f"Cryptocurrencies: {', '.join(cryptos_for_year)}")
        
        # Fetch all data for the period
        prices_df, returns_df = fetch_and_process_data(cryptos_for_year, start_date, end_date)
        
        # Generate signals for each cryptocurrency
        signals_dict = {}
        
        for crypto in cryptos_for_year:
            if crypto in returns_df.columns:
                try:
                    returns = returns_df[crypto]
                    if len(returns) < MODEL_PARAMS['lookback_window']:
                        print(f"Not enough data for {crypto} in period {period_label}. Skipping.")
                        continue
                    
                    # Detect changepoints in the return series
                    changepoints = detect_changepoints(returns, lookback_window=MODEL_PARAMS['lookback_window'])
                    
                    # Generate trading signals using the GPR+LSTM model
                    positions = generate_signals(
                        returns, changepoints,
                        train_ratio=MODEL_PARAMS['train_ratio'],
                        epochs=MODEL_PARAMS['epochs'],
                        dropout_rate=MODEL_PARAMS['dropout_rate'],
                        lstm_hidden_units=MODEL_PARAMS['lstm_hidden_units'],
                        learning_rate=MODEL_PARAMS['learning_rate'],
                        batch_size=MODEL_PARAMS['batch_size']
                    )
                    
                    signals_dict[crypto] = positions
                except Exception as e:
                    print(f"Error generating signals for {crypto}: {e}")
        
        # Backtest with equal weight allocation
        print("\nRunning equal-weight portfolio backtest:")
        portfolio, asset_portfolios = backtest_equal_weight_portfolio(
            prices_df, signals_dict, fee_rate=TRADING_PARAMS['fee_rate']
        )
        
        if portfolio:
            # Record main portfolio results
            portfolio_stats = portfolio.stats()
            result = {
                "period": period_label,
                "strategy": "Equal Weight Portfolio",
                "num_assets": len(signals_dict),
                **{key: str(value) if pd.isna(value) else value for key, value in portfolio_stats.items()}
            }
            results.append(result)
            
            # Also record individual asset results
            for crypto, pf in asset_portfolios.items():
                asset_stats = pf.stats()
                asset_result = {
                    "period": period_label,
                    "crypto": crypto,
                    "strategy": "Individual Asset",
                    **{key: str(value) if pd.isna(value) else value for key, value in asset_stats.items()}
                }
                results.append(asset_result)
        
        # Backtest with volatility-adjusted weights
        print("\nRunning volatility-adjusted portfolio backtest:")
        vol_portfolio, vol_asset_portfolios = backtest_volatility_adjusted_portfolio(
            prices_df, signals_dict, current_year, fee_rate=TRADING_PARAMS['fee_rate']
        )
        
        if vol_portfolio:
            # Record volatility-adjusted portfolio results
            vol_portfolio_stats = vol_portfolio.stats()
            vol_result = {
                "period": period_label,
                "strategy": "Volatility-Adjusted Portfolio",
                "num_assets": len(signals_dict),
                **{key: str(value) if pd.isna(value) else value for key, value in vol_portfolio_stats.items()}
            }
            results.append(vol_result)

    # Save all summary results into a DataFrame
    results_df = pd.DataFrame(results)
    
    # Convert numeric columns to float for proper aggregation
    numeric_cols = ['total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate']
    for col in numeric_cols:
        if col in results_df.columns:
            results_df[col] = pd.to_numeric(results_df[col], errors='coerce')
    
    print("\nBacktest Results:")
    print(results_df)
    
    # Save the DataFrame to a CSV file
    results_df.to_csv(RESULTS_FILE, index=False)
    
    # Save volatility-adjusted results to a separate file
    vol_results = results_df[results_df["strategy"] == "Volatility-Adjusted Portfolio"]
    if not vol_results.empty:
        vol_results.to_csv(VOL_RESULTS_FILE, index=False)
    
    # Calculate and display yearly performance comparison
    portfolio_results = results_df[(results_df["strategy"] == "Equal Weight Portfolio") | 
                                  (results_df["strategy"] == "Volatility-Adjusted Portfolio")]
    if not portfolio_results.empty:
        print("\nPortfolio Performance by Year and Strategy:")
        
        # Create a cleaner pivot table for display
        pivot_display = portfolio_results.pivot(
            index="period", 
            columns="strategy", 
            values=["total_return", "sharpe_ratio", "max_drawdown"]
        )
        print(pivot_display)
        
        # Calculate average metrics by strategy - correctly handling the structure
        # Use the original dataframe for calculating means to avoid MultiIndex issues
        avg_metrics = portfolio_results.groupby("strategy")[numeric_cols].mean()
        print("\nAverage Performance by Strategy:")
        print(avg_metrics)
        
        # Plot comparison of strategies
        try:
            import matplotlib.pyplot as plt
            
            # Plot total returns by strategy
            plt.figure(figsize=(12, 6))
            
            # Plot bar chart comparing strategies
            pivot_data = portfolio_results.pivot(index="period", columns="strategy", values="total_return")
            pivot_data.plot(kind='bar', figsize=(12, 6))
            plt.title('Total Return Comparison by Strategy')
            plt.ylabel('Total Return')
            plt.xlabel('Period')
            plt.tight_layout()
            plt.savefig('Backtested/strategy_comparison.png')
            plt.close()
            
            # Also create a comparative line chart for cumulative performance
            plt.figure(figsize=(14, 7))
            
            # Group by strategy and calculate cumulative performance
            for strategy in portfolio_results['strategy'].unique():
                strategy_data = portfolio_results[portfolio_results['strategy'] == strategy]
                periods = strategy_data['period'].tolist()
                returns = strategy_data['total_return'].tolist()
                
                # Generate cumulative returns (1 + return)
                cumulative_returns = [1]
                for ret in returns:
                    cumulative_returns.append(cumulative_returns[-1] * (1 + ret))
                
                # Remove the initial 1
                cumulative_returns.pop(0)
                
                # Plot the cumulative returns line
                plt.plot(periods, cumulative_returns, marker='o', linewidth=2, label=strategy)
            
            plt.title('Cumulative Performance by Strategy')
            plt.ylabel('Cumulative Return (starting with 1.0)')
            plt.xlabel('Period')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.legend()
            plt.tight_layout()
            plt.savefig('Backtested/cumulative_performance.png')
            plt.close()
            
            print("\nGenerated comparison charts saved to 'Backtested/' directory")
        except Exception as e:
            print(f"Error generating comparison chart: {e}")

if __name__ == "__main__":
    main()


Processing period 2019 (2019-01-01 to 2019-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, BCH-USD, EOS-USD, LTC-USD, XLM-USD, ADA-USD, TRX-USD, BSV-USD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Running equal-weight portfolio backtest:


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Running volatility-adjusted portfolio backtest:
Calculating volatility from 2018-01-01 to 2019-01-01
BTC-USD annual volatility: 0.6739
ETH-USD annual volatility: 0.8904


...


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


ADA-USD annual volatility: 0.6882
DOGE-USD annual volatility: 0.8559
SOL-USD annual volatility: 0.6796


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


DOT-USD annual volatility: 0.7003
LTC-USD annual volatility: 0.6283
AVAX-USD annual volatility: 0.7819

Volatility-adjusted weights:
BTC-USD: 0.1409
BNB-USD: 0.1285
ETH-USD: 0.1157
LTC-USD: 0.0996
SOL-USD: 0.0921
ADA-USD: 0.0909
XRP-USD: 0.0898
DOT-USD: 0.0894
AVAX-USD: 0.0800
DOGE-USD: 0.0731

Backtest Results:
      period                       strategy  num_assets  total_return  \
0       2019         Equal Weight Portfolio        10.0      0.415276   
1       2019               Individual Asset         NaN           NaN   
2       2019               Individual Asset         NaN           NaN   
3       2019               Individual Asset         NaN           NaN   
4       2019               Individual Asset         NaN           NaN   
..       ...                            ...         ...           ...   
79  2025_YTD               Individual Asset         NaN           NaN   
80  2025_YTD               Individual Asset         NaN           NaN   
81  2025_YTD               In

<Figure size 1200x600 with 0 Axes>

In [4]:
df_res = pd.read_csv(RESULTS_FILE)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_res

,period,strategy,num_assets,total_return,sharpe_ratio,max_drawdown,win_rate,crypto,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,2019,Equal Weight Portfolio,10.0,0.415276,1.342450,-0.098591,0.325069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,BTC-USD,2019-01-02,2019-12-30,363 days,100.0,184.526728,84.526728,84.941362,100.000000,2.440565e+01,22.717278,89 days 00:00:00,81.0,81.0,0.0,0.000000,33.333333,20.367978,-13.196963,6.456962,-1.863916,3 days 08:53:20,1 days 05:46:40,1.533479,1.043540e+00,1.512869,3.748276,1.434426,2.593118
2,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,ETH-USD,2019-01-02,2019-12-30,363 days,100.0,134.536238,34.536238,-14.456327,100.000000,2.297389e+01,24.640693,84 days 00:00:00,93.0,93.0,0.0,0.000000,27.956989,23.611161,-9.071376,7.211959,-2.142282,3 days 06:27:41.538461538,1 days 07:09:51.044776119,1.189047,3.713574e-01,0.810286,1.410525,1.194302,1.361134
3,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,XRP-USD,2019-01-02,2019-12-30,363 days,100.0,99.913906,-0.086094,-48.173852,0.474620,5.851888e-02,0.135976,191 days 00:00:00,94.0,94.0,0.0,0.000000,25.531915,28.723192,-9.132107,5.631823,-2.255973,3 days 07:00:00,1 days 06:10:17.142857142,0.831963,-9.158929e-04,-0.491249,-0.636643,0.889674,-0.802439
4,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,BCH-USD,2019-01-02,2019-12-30,363 days,100.0,155.864808,55.864808,21.869270,100.000000,2.670625e+01,49.644043,229 days 00:00:00,95.0,95.0,0.0,0.000000,34.736842,70.684594,-10.840778,8.003329,-2.967819,2 days 18:10:54.545454545,1 days 03:29:01.935483870,1.206964,5.880506e-01,0.922421,1.132994,1.269834,1.974964
5,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,EOS-USD,2019-01-02,2019-12-30,363 days,100.0,100.279441,0.279441,-8.383521,8.239731,7.432204e-01,3.152220,216 days 00:00:00,95.0,95.0,0.0,0.000000,27.368421,35.625649,-15.128080,8.020575,-2.852328,3 days 02:46:09.230769230,1 days 09:02:36.521739130,1.034786,2.941484e-03,0.106676,0.089138,1.025488,0.166333
6,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,LTC-USD,2019-01-02,2019-12-30,363 days,100.0,178.086360,78.086360,27.857105,76.620953,1.167379e+01,12.226189,184 days 00:00:00,86.0,86.0,0.0,0.000000,39.534884,51.882708,-10.483983,7.750090,-2.550088,2 days 21:52:56.470588235,1 days 03:41:32.307692307,1.870322,9.079809e-01,1.860753,6.433199,1.569412,3.836979
7,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,XLM-USD,2019-01-02,2019-12-30,363 days,100.0,100.027810,0.027810,-61.490309,0.141435,1.585600e-02,0.037996,165 days 00:00:00,90.0,90.0,0.0,0.000000,28.888889,40.713323,-8.917515,7.952493,-2.464179,3 days 04:36:55.384615384,1 days 08:37:30,1.192415,3.090001e-04,0.472987,0.735949,1.116672,0.821676
8,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,ADA-USD,2019-01-02,2019-12-30,363 days,100.0,100.005277,0.005277,-25.772238,0.097697,9.872293e-03,0.029751,201 days 00:00:00,90.0,90.0,0.0,0.000000,30.000000,20.572796,-8.426702,6.538109,-2.734295,3 days 03:33:20,1 days 11:02:51.428571428,1.053361,5.863046e-05,0.128577,0.178344,1.028826,0.207654
9,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,TRX-USD,2019-01-02,2019-12-30,363 days,100.0,100.010100,0.010100,-33.735540,0.038551,4.062208e-03,0.010006,211 days 00:00:00,91.0,91.0,0.0,0.000000,34.065934,27.588414,-9.776020,6.412079,-2.548859,2 days 23:13:32.903225806,1 days 08:24:00,1.285987,1.109867e-04,0.607473,1.014959,1.141593,0.984136


In [5]:
df_res[df_res['num_assets'] == 10.0]

,period,strategy,num_assets,total_return,sharpe_ratio,max_drawdown,win_rate,crypto,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,2019,Equal Weight Portfolio,10.0,0.415276,1.342450,-0.098591,0.325069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2019,Volatility-Adjusted Portfolio,10.0,0.401634,1.289522,-0.101842,0.336088,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2020,Equal Weight Portfolio,10.0,0.485495,1.326777,-0.121511,0.420330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2020,Volatility-Adjusted Portfolio,10.0,0.489263,1.507597,-0.123128,0.425824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2021,Equal Weight Portfolio,10.0,0.364744,0.963383,-0.302303,0.421488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2021,Volatility-Adjusted Portfolio,10.0,0.435235,1.038179,-0.315660,0.413223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,2022,Equal Weight Portfolio,10.0,-0.162157,-0.613678,-0.265477,0.327824,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2022,Volatility-Adjusted Portfolio,10.0,-0.189139,-0.586290,-0.308763,0.330579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,2023,Equal Weight Portfolio,10.0,0.118782,0.813948,-0.104160,0.358127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,2023,Volatility-Adjusted Portfolio,10.0,0.148528,0.873080,-0.120719,0.355372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
